### 作業目的: 使用樹型模型進行文章分類

本次作業主利用[Amazon Review data中的All Beauty](https://nijianmo.github.io/amazon/index.html)來進行review評價分類(文章分類)

資料中將review分為1,2,3,4,5分，而在這份作業，我們將評論改分為差評價、普通評價、優良評價(1,2-->1差評、3-->2普通評價、4,5-->3優良評價)

### 載入套件

In [1]:
import json
import re
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

### 資料前處理
文本資料較為龐大，這裡我們取前10000筆資料來進行作業練習

In [2]:
#load json data
all_reviews = []
with open('All_Beauty.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        all_reviews.append(json.loads(line))
all_reviews[0]

{'overall': 1.0,
 'verified': True,
 'reviewTime': '02 19, 2015',
 'reviewerID': 'A1V6B6TNIC10QE',
 'asin': '0143026860',
 'reviewerName': 'theodore j bigham',
 'reviewText': 'great',
 'summary': 'One Star',
 'unixReviewTime': 1424304000}

In [3]:
#parse label(overall) and corpus(reviewText)
corpus = []
labels = []

for review in all_reviews[:10000]:
    if review.get("reviewText", False) and review.get("overall", False):
        corpus.append(review["reviewText"])
        labels.append(review["overall"])
        
#transform labels: 1,2 --> 1 and 3 --> 2 and 4,5 --> 3
for i, label in enumerate(labels):
    if label == 1 or label == 2:
        labels[i] = 1
    elif label == 3:
        labels[i] = 2
    else:
        labels[i] = 3

In [4]:
from nltk.corpus import stopwords

import nltk

nltk.download('stopwords')

# Lemmatize with POS Tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 

## 創建Lemmatizer
lemmatizer = WordNetLemmatizer() 
def get_wordnet_pos(word):
    """將pos_tag結果mapping到lemmatizer中pos的格式"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


def clean_content(X):
    stop_words = set(stopwords.words('english'))
    content_clean = []
    # remove non-alphabet characters
    pattern = r"\S*@\S*|\\n|[^a-zA-Z0-9 ]"
    X_clean = re.sub(pattern,' ', X).lower()
    # tokenize
    X_word_tokenize = nltk.word_tokenize(X_clean)

    # stopwords_lemmatizer
    for word in X_word_tokenize:
        if word not in stop_words:
            word = lemmatizer.lemmatize(word, get_wordnet_pos(word))
            content_clean.append(word)
                
    return ' '.join(content_clean)

[nltk_data] Downloading package stopwords to C:\Users\Guan-Ting
[nltk_data]     Chen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
#split corpus and label into train and test
x_train, x_test, y_train, y_test = train_test_split(corpus, labels, test_size=0.2, random_state=0)
len(x_train), len(x_test), len(y_train), len(y_test)

(7996, 1999, 7996, 1999)

In [6]:
#change corpus into vector
#you can use tfidf or BoW here
tf = TfidfVectorizer()
tf.fit(x_train)

#transform training and testing corpus into vector form
x_train = tf.transform(x_train).toarray()
x_test = tf.transform(x_test).toarray()

### 訓練與預測

In [7]:
# #build classification model (decision tree, random forest, or adaboost)
# #start training
# cls = RandomForestClassifier(n_estimators=200,
#                              class_weight={1: 15, 2: 25, 3:1})
# cls.fit(x_train, y_train)

cls = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=6, min_samples_split=2),
                         n_estimators=100)

cls.fit(x_train, y_train)



AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='gini',
                                                         max_depth=6,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=None,
                             

In [8]:
#start inference
y_pred = cls.predict(x_test)

In [9]:
#calculate accuracy
print('Accuracy:', sum(y_test==y_pred) / len(y_pred))

Accuracy: 0.9044522261130565


In [10]:
#calculate confusion matrix, precision, recall, and f1-score
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.61      0.34      0.44       134
           2       0.09      0.01      0.02        73
           3       0.92      0.98      0.95      1792

    accuracy                           0.90      1999
   macro avg       0.54      0.45      0.47      1999
weighted avg       0.87      0.90      0.88      1999

[[  46    2   86]
 [   7    1   65]
 [  23    8 1761]]


由上述資訊可以發現, 模型在好評的準確度高(precision, recall都高), 而在差評的部分表現較不理想, 在普通評價的部分大部分跟差評搞混,
同學可以試著學習到的各種方法來提升模型的表現